# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import torch
import logging

sys.path.append("../")
import src.utils.logging_utils as logging_utils
import src.functional as functional
import src.models as models
import src.tokens as tokens
import src.dataset as dataset
import src.patchscope_utils as patchscope_utils
import proto.patchscope_pb2 as patchscope_pb2

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

2024-10-25 20:28:35 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.2-3B"
MODEL_KEY = "meta-llama/Llama-3.1-8B"

# MODEL_KEY = "google/gemma-2-9b-it"
# MODEL_KEY = "google/gemma-2-2b"

mt = models.ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.bfloat16,
)

mt.n_layer

2024-10-25 20:28:35 src.models WARNING  meta-llama/Llama-3.1-8B not found in models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2024-10-25 20:28:37 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-10-25 20:30:02 src.models INFO     loaded model <meta-llama/Llama-3.1-8B> | size: 15316.516 MB | dtype: torch.bfloat16 | device: cuda:0


32

# Demo

In [4]:
# true_prompt = "The city of Paris is in the country of France."
# false_prompt = "The city of Paris is in the country of Italy."
# true_input, false_input = [tokens.prepare_input(p, mt) for p in (true_prompt, false_prompt)]

In [5]:
# # layers = [24]
# layers = list(range(3, 30))

# true_h, false_h = [
#     patchscope_utils.get_h_layers(mt, input_, layers) for input_ in (true_input, false_input)
# ]

In [6]:
# true_token, false_token = [mt.tokenizer.encode(t)[-1] for t in ("True", "False")]
# [mt.tokenizer.decode(t) for t in (true_token, false_token)]

In [7]:
# for prompt, h in zip((true_prompt, false_prompt), (true_h, false_h)):
#     print(prompt)
#     _, result_dict = functional.patchscope(
#         mt = mt, 
#         hs = patchscope_utils.get_h_with_target_layer(h, 3) if len(h) == 1 else h,
#         target_prompt = target_prompt,
#         interested_tokens = (true_token, false_token),
#         k = 5)
#     for t in (true_token, false_token):
#         print("   ", result_dict[t])


In [8]:
# gmt_dataset = dataset.GMTDataset.from_csv("cities.csv", few_shot=False).examples
# len(gmt_dataset)

# Evaluation

In [9]:
base_prompt = '''The city of Tokyo is in Japan. This statement is: True
The city of Hanoi is in Poland. This statement is: False
placeholder placeholder placeholder placeholder placeholder This statement is: '''

instruct_prompt = '''<|start_header_id|>user<|end_header_id|>

True or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''

target_prompt = {
    "meta-llama/Llama-3.2-3B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.1-8B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.2-3B" : base_prompt,
    "meta-llama/Llama-3.1-8B" : base_prompt,
}[MODEL_KEY]

model_short_name = {
    "meta-llama/Llama-3.2-3B-Instruct" : "llama_3b_instruct",
    "meta-llama/Llama-3.1-8B-Instruct" : "llama_8b_instruct",
    "meta-llama/Llama-3.2-3B" : "llama_3b",
    "meta-llama/Llama-3.1-8B" : "llama_8b",
}[MODEL_KEY]

In [ ]:
print(MODEL_KEY, model_short_name)

meta-llama/Llama-3.1-8B llama_8b


In [11]:
result_set_name = model_short_name + "__all_layers"
evaluation_results = patchscope_pb2.EvaluationResults()

for filename in dataset.GMT_DATA_FILES:
    examples = dataset.GMTDataset.simple_get_examples(filename)[:100]
    evaluation_config = patchscope_pb2.EvaluationConfig(
        model_key=MODEL_KEY,
        dataset=filename,
        target_prompt=target_prompt,
        label_to_token={ "1": "True", "0": "False" },
        patchscope_config=patchscope_pb2.PatchscopeConfig(
            source_layers=list(range(mt.n_layer)),
            target_layers=[]
        )
    )
    evaluation_runner = patchscope_utils.EvaluationRunner(evaluation_config, mt)
    evaluation_result = evaluation_runner.evaluate(examples)
    evaluation_result.result_set_name = result_set_name
    # evaluation_result = evaluation_runner.evaluate([(true_prompt, True), (false_prompt, False)])
    evaluation_results.results.append(evaluation_result)
    print(filename, evaluation_result.accuracy)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:37<00:00,  1.57s/it]


sp_en_trans.csv 0.41


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:36<00:00,  1.57s/it]


neg_sp_en_trans.csv 0.42


...

common_claim_true_false.csv 0.54


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:32<00:00,  1.53s/it]


companies_true_false.csv 0.63


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:32<00:00,  1.52s/it]

counterfact_true_false.csv 0.57


In [12]:
with open(f"results/{result_set_name}.binpb", "wb") as f:
    f.write(evaluation_results.SerializeToString())